## ６章整然データをつくる
この章ではデータの形式を変えるreshapingについて次の事項を学ぶ。
- 列を行に変える unpivot/melt/gather
- 行を列に変える pivot/cast/spread
- データを正規化するためにDataFrameを複数の表に分割する
- 複数のパートからデータを集める

### 複数列に値が入っているとき
横持ちから縦持ちへの変換

In [1]:
import pandas as pd
pew = pd.read_csv("./data/pew.csv")

In [2]:
pew.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27.0,34.0,60.0,81.0,76.0,137.0,122.0,109.0,84.0,96.0
1,Atheist,12.0,27.0,37.0,52.0,35.0,70.0,73.0,59.0,74.0,76.0
2,Buddhist,27.0,21.0,30.0,34.0,33.0,58.0,62.0,39.0,53.0,54.0
3,Catholic,418.0,617.0,732.0,670.0,638.0,1116.0,949.0,792.0,633.0,1489.0
4,Don’t know/refused,15.0,14.0,15.0,11.0,10.0,35.0,21.0,17.0,18.0,116.0


上記の値を見ると収入に関する値が複数列に別れている。このデータを宗教と収入と度数の変数を持つように変換する。このようなデータビューは横持ちと呼ばれる。これを整然とした縦持ちに変換するにはunpivotかmeltかgatherを行う必要がある。（どの用語を使うかはプログラミング言語に依存する）  
pandasではmelt関数でフォーマットを変換できる。

In [3]:
pew_long = pd.melt(pew, id_vars="religion")
pew_long.head()

,religion,variable,value
0,Agnostic,<$10k,27.0
1,Atheist,<$10k,12.0
2,Buddhist,<$10k,27.0
3,Catholic,<$10k,418.0
4,Don’t know/refused,<$10k,15.0


In [4]:
# 列名変更
pew_long = pd.melt(pew, id_vars="religion"
                  ,var_name="income"
                  ,value_name="count")
pew_long.head()

,religion,income,count
0,Agnostic,<$10k,27.0
1,Atheist,<$10k,12.0
2,Buddhist,<$10k,27.0
3,Catholic,<$10k,418.0
4,Don’t know/refused,<$10k,15.0


### 複数の列を残す
ここではweekの列を横持ちしている。

In [5]:
billboard = pd.read_csv("./data/billboard.csv")
billboard.head()

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
billboard_long = pd.melt(
    billboard
    ,id_vars=["year","artist","track","time","date.entered"]
    ,var_name="week"
    ,value_name="rating")
billboard_long.head()

,year,artist,track,time,date.entered,week,rating
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk1,91.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk1,81.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk1,57.0


### 複数の変数を含む列がある場合
列名Cases_GuineaとDeaths_Guineaには国名と患者の数、死者の数が情報として含まれている。このデータも横持ちなので融解する。

In [7]:
ebola = pd.read_csv("./data/country_timeseries.csv")
ebola.head()

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
0,1/5/2015,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN,1786.0,NaN,2977.0,NaN,NaN,NaN,NaN,NaN
1,1/4/2015,288,2775.0,NaN,9780.0,NaN,NaN,NaN,NaN,NaN,1781.0,NaN,2943.0,NaN,NaN,NaN,NaN,NaN
2,1/3/2015,287,2769.0,8166.0,9722.0,NaN,NaN,NaN,NaN,NaN,1767.0,3496.0,2915.0,NaN,NaN,NaN,NaN,NaN
3,1/2/2015,286,NaN,8157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3496.0,NaN,NaN,NaN,NaN,NaN,NaN
4,12/31/2014,284,2730.0,8115.0,9633.0,NaN,NaN,NaN,NaN,NaN,1739.0,3471.0,2827.0,NaN,NaN,NaN,NaN,NaN


In [8]:
ebola_long = pd.melt(ebola
                    ,id_vars=["Date","Day"])
ebola_long.head()

,Date,Day,variable,value
0,1/5/2015,289,Cases_Guinea,2776.0
1,1/4/2015,288,Cases_Guinea,2775.0
2,1/3/2015,287,Cases_Guinea,2769.0
3,1/2/2015,286,Cases_Guinea,NaN
4,12/31/2014,284,Cases_Guinea,2730.0


### 列を分割して追加する単純な方法
アイデアとしては、該当の列名のアンダースコアで分割する方法が考えられる。

In [9]:
# 変数の列名を取得し、デリミタによる分割
variable_split = ebola_long.variable.str.split("_")
variable_split[:5]

0    [Cases, Guinea]
1    [Cases, Guinea]
2    [Cases, Guinea]
3    [Cases, Guinea]
4    [Cases, Guinea]
Name: variable, dtype: object

In [10]:
# 上記を新しい列に割り当てる
status_v = variable_split.str.get(0)
country_v = variable_split.str.get(1)
print(status_v[:5])
print(country_v[:5])

0    Cases
1    Cases
2    Cases
3    Cases
4    Cases
Name: variable, dtype: object
0    Guinea
1    Guinea
2    Guinea
3    Guinea
4    Guinea
Name: variable, dtype: object


In [11]:
ebola_long["status"] = status_v
ebola_long["country"] = country_v
ebola_long.head()

,Date,Day,variable,value,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea


### 分割と結合を一度に行う（単純な方法）

In [12]:
variable_split = ebola_long.variable.str.split("_", expand=True)
variable_split.columns = ["status", "country"]
ebola_parsed = pd.concat([ebola_long,variable_split], axis=1)
ebola_parsed.head()

,Date,Day,variable,value,status,country,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea,Cases,Guinea


### 分割と結合を一度に行う（複雑な方法）

In [13]:
ebola_long["status"], ebola_long["country"] = zip(*ebola_long.variable.str.split("_"))
ebola_long.head()

,Date,Day,variable,value,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea


### 行と列の両方に変数があるとき

In [14]:
weather = pd.read_csv("./data/weather.csv")
weather.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
weather_melt = pd.melt(weather
                      ,id_vars=["id","year","month","element"]
                      ,var_name="day"
                      ,value_name="temp")
weather_melt.head()

,id,year,month,element,day,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


In [16]:
# elementをpivot
weather_tidy = weather_melt.pivot_table(
    index=["id","year","month","day"]
    ,columns="element"
    ,values="temp")
weather_tidy.head()

element                 tmax  tmin
id      year month day            
MX17004 2010 1     d30  27.8  14.5
             2     d11  29.7  13.4
                   d2   27.3  14.4
                   d23  29.9  10.7
                   d3   24.1  14.4

In [17]:
# 平坦化
weather_tidy_flat = weather_tidy.reset_index()
weather_tidy_flat.head()

element,id,year,month,day,tmax,tmin
0,MX17004,2010,1,d30,27.8,14.5
1,MX17004,2010,2,d11,29.7,13.4
2,MX17004,2010,2,d2,27.3,14.4
3,MX17004,2010,2,d23,29.9,10.7
4,MX17004,2010,2,d3,24.1,14.4


### 正規化したいとき
次のデータではトラック情報に重複があるため、別テーブルで管理する。

In [20]:
billboard_long[billboard_long.track == "Loser"].head()

,year,artist,track,time,date.entered,week,rating
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
320,2000,3 Doors Down,Loser,4:24,2000-10-21,wk2,76.0
637,2000,3 Doors Down,Loser,4:24,2000-10-21,wk3,72.0
954,2000,3 Doors Down,Loser,4:24,2000-10-21,wk4,69.0
1271,2000,3 Doors Down,Loser,4:24,2000-10-21,wk5,67.0


In [21]:
billboard_song = billboard_long[["year","artist","track","time"]]
billboard_song.shape

(24092, 4)

In [22]:
# 重複を削除
billboard_song = billboard_song.drop_duplicates()
billboard_song.shape

(317, 4)

In [24]:
# ユニークID設定
billboard_song["id"] = range(len(billboard_song))
billboard_song.head()

,year,artist,track,time,id
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,1
2,2000,3 Doors Down,Kryptonite,3:53,2
3,2000,3 Doors Down,Loser,4:24,3
4,2000,504 Boyz,Wobble Wobble,3:35,4


In [26]:
# マージ
billboard_ratings = billboard_long.merge(
    billboard_song, on=["year","artist","track","time"]
)
billboard_ratings.head()

,year,artist,track,time,date.entered,week,rating,id
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0,0
1,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk2,82.0,0
2,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk3,72.0,0
3,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk4,77.0,0
4,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk5,87.0,0


In [28]:
billboard_ratings = billboard_ratings[["id","date.entered","week","rating"]]
billboard_ratings.head()

,id,date.entered,week,rating
0,0,2000-02-26,wk1,87.0
1,0,2000-02-26,wk2,82.0
2,0,2000-02-26,wk3,72.0
3,0,2000-02-26,wk4,77.0
4,0,2000-02-26,wk5,87.0
